<div style="font-size: 22pt; font-weight: bold; text-align: center;
    border: 3px solid #666; border-radius: 6px; 
    padding: 10px; margin: 20px 0;
    background-color: #fafafa;">
    Google Earth Engine<br>
    <span style="font-size: 16pt; color: #666;">Getting Started Guide</span>
</div>

<div style="background-color: #f8f9fa; border: 1px solid #dee2e6; border-radius: 6px; 
    padding: 15px; margin: 10px 0; font-size: 12pt; color: #495057; 
    font-family: Arial, sans-serif;">
    <div style="text-align: center; margin-bottom: 10px;">
        <strong>Author:</strong> Dr. Saad Laouadi<br>
        <em>Educational Content for Geospatial Analysis</em>
    </div>
    <hr style="border: none; border-top: 1px solid #dee2e6; margin: 10px 0;">
    <div style="font-size: 11pt; line-height: 1.4;">
        <strong>Usage Terms:</strong><br>
        • This content is provided for educational and research purposes<br>
        • Please provide attribution when using or sharing this material<br>
        • Commercial use requires prior permission - contact the author<br>
        • Feel free to adapt for non-commercial educational use
    </div>
</div>

In [1]:
#************************************************************
#*           GOOGLE EARTH ENGINE - FIRST STEPS              *
#*                  Essential Libraries                     *
#************************************************************

# Import the working tools
import ee 
import geemap
import ipywidgets as widgets
from IPython.display import display

print("All libraries imported successfully!")

print(f"   • Earth Engine: {ee.__version__}")
print(f"   • Geemap: {geemap.__version__}")
print(f"   • IPython Widgets: {widgets.__version__}")

All libraries imported successfully!
   • Earth Engine: 1.6.6
   • Geemap: 0.36.2
   • IPython Widgets: 8.1.7


In [2]:
# Initialize Google Earth Engine
try:
    # Replace 'your-project-id' with your actual project ID
    ee.Initialize(project='your-project-id')
    print("Google Earth Engine initialized successfully!")

except Exception as e:
    print("Authentication required!")
    print("Please run the following command first:")
    print("ee.Authenticate()")
    print(f"\n Error details: {str(e)}")
    raise

Google Earth Engine initialized successfully!


In [3]:
# Create an interactive map centered on Japan
# Try geemap (if leaflet extension works)

Map = geemap.Map(
    center=[35.6762, 139.6503],  # Tokyo, Japan coordinates
    zoom=6,                      # Zoom level to show more of Japan
    height='600px'
)

print(type(Map))
print("Geemap interactive map created successfully!")
# display(Map)           # This did not work for me due to some issue with leaflet package
                         # you may need to uncommend if you are using classic jupyter notebook
                         # It will work just fine.

<class 'geemap.geemap.Map'>
Geemap interactive map created successfully!


In [4]:
# Using folium to display map
# Since I am using Jupyterlab, I had to fall back to folium package
import folium

# Create folium map as backup
folium_map = folium.Map(
    location=[36.7538, 3.0588],  # Algiers, Algeria
    zoom_start=8,
    width='100%',
    height='60%'
)
print("🗺️ Folium map created as alternative")
print("📍 Centered on Algiers, Algeria")

# Display folium map (works without extensions)
folium_map

🗺️ Folium map created as alternative
📍 Centered on Algiers, Algeria


---

In [5]:
# Load a Sentinel-2 image
# We'll use Algeria (Algiers) as our area of interest
algeria = ee.Geometry.Point([3.0588, 36.7538])      # Algiers, Algeria coordinates

In [6]:
# Get a cloud-free Sentinel-2 image
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(algeria) \
    .filterDate('2024-01-01', '2024-03-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .select(['B4', 'B3', 'B2', 'B8', 'B11']) \
    .median()

# Calculate NDVI
ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Print information about the processed images
print("Successfully processed Sentinel-2 data for Algeria")
print("Available bands:", sentinel2.bandNames().getInfo())
print("Area bounds:", algeria.buffer(50000).bounds().getInfo())

Successfully processed Sentinel-2 data for Algeria
Available bands: ['B4', 'B3', 'B2', 'B8', 'B11']
Area bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[2.5022159484946354, 36.30402860761232], [3.6178948848610375, 36.30402860761232], [3.6178948848610375, 37.2037432975741], [2.5022159484946354, 37.2037432975741], [2.5022159484946354, 36.30402860761232]]]}


In [7]:
# Get NDVI statistics
ndvi_stats = ndvi.reduceRegion(
    reducer=ee.Reducer.minMax().combine(ee.Reducer.mean(), sharedInputs=True),
    geometry=algeria.buffer(50000),
    scale=10,
    maxPixels=1e9
)
print("NDVI Statistics:", ndvi_stats.getInfo())

NDVI Statistics: {'NDVI_max': 1, 'NDVI_mean': 0.2549056428877465, 'NDVI_min': -1}


In [8]:
# Export to Google Drive
# *********************
def export_to_drive():
    # Export true color image
    task1 = ee.batch.Export.image.toDrive(
        image=sentinel2.select(['B4', 'B3', 'B2']),
        description='Oman_TrueColor',
        scale=10,
        region=oman.buffer(25000),
        fileFormat='GeoTIFF'
    )
    
    # Export NDVI
    task2 = ee.batch.Export.image.toDrive(
        image=ndvi,
        description='Oman_NDVI',
        scale=10,
        region=oman.buffer(25000),
        fileFormat='GeoTIFF'
    )
    
    # Start tasks
    task1.start()
    task2.start()
    
    print("Export tasks started. Check your Google Drive in a few minutes.")
    print(f" Task 1 ID: {task1.id}")
    print(f" Task 2 ID: {task2.id}")

In [11]:
# Generate static map
# *******************
def generate_static_map():
    # Create a static map URL
    url = sentinel2.select(['B4', 'B3', 'B2']).getThumbURL({
        'min': 0,
        'max': 3000,
        'dimensions': 1024,
        'region': algeria.buffer(50000),
        'format': 'png'
    })
    print("Static map URL (copy to browser):")
    print(url)

# Run alternatives
print("\n" + "="*50)
print("Choose an option:")
print("1. Export to Google Drive: export_to_drive()")
print("2. Generate static map URL: generate_static_map()")
print("="*50)


Choose an option:
1. Export to Google Drive: export_to_drive()
2. Generate static map URL: generate_static_map()


In [12]:
# Uncomment to use:
# export_to_drive()
generate_static_map()

Static map URL (copy to browser):
https://earthengine.googleapis.com/v1/projects/learngee-471509/thumbnails/879a59ead7151e30f8b47c2a4d11589c-a25e2a7daee4e91beb0ac9ae44db66f3:getPixels
